# Fine-tuning a model on a token classification task

## Loading the dataset

## Preprocessing the data

In [1]:
from transformers import AutoTokenizer
import os
import optuna
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, AutoConfig
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np
from datasets import load_from_disk, concatenate_datasets
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score

tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Inicializando a variável particoes
particoes = {}
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
# atch_size = 8
print(particoes.keys())

label_all_tokens = True
# Carregar o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Caminho correto para as partições
base_path = "../models/"

# Carregar as partições diretamente do caminho especificado
for i in range(10):
    partition_path = f"{base_path}data_division_{i}"
    print(f"Verificando partição: {partition_path}")  # Para depuração
    
    if os.path.exists(partition_path):  # Verifica se o diretório existe
        try:
            particoes[f"particao_{i}"] = load_from_disk(partition_path)
        except Exception as e:
            print(f"Erro ao carregar a partição {partition_path}: {e}")
    else:
        print(f"Partição {partition_path} não encontrada!")  # Alerta se o diretório não existir  # Verificar quais partições foram carregadas corretamente
print("Partições carregadas:", list(particoes.keys()))


# Criar o mapeamento de label2id
label2id = {
    'O': 0,                    # Fora de entidade
    'B-LOCAL': 1,               # Início da entidade DATA
    'I-LOCAL': 2,               # Interior da entidade DATA
    'B-ORGANIZACAO': 3,             # Início da entidade EVENTO
    'I-ORGANIZACAO': 4,             # Interior da entidade EVENTO
    'B-PESSOA': 5,        # Início da entidade FUNDapelido
    'I-PESSOA': 6,        # Interior da entidade FUNDapelido
    'B-TEMPO': 7,        # Início da entidade FUNDapelido
    'I-TEMPO': 8, 
}


# Exibir o mapeamento de labels
print(f"Label2ID mapping: {label2id}")

def tokenize_and_align_labels(examples):
    # Verificando se 'sentences' está presente nos dados
    if "sentences" not in examples:
        raise KeyError("A chave 'sentences' não foi encontrada nos dados.")

    tokenized_inputs = []
    labels = []

    for sentence in examples["sentences"]:
        # Tokenizando os tokens de cada sentença
        tokenized_sentence = tokenizer(sentence["tokens"], truncation=True, is_split_into_words=True, max_length=512)

        word_ids = tokenized_sentence.word_ids()  # Pega os ids dos tokens
        sentence_labels = sentence["labels"]  # Labels da sentença

        previous_word_idx = None
        label_ids = []          # Atribuindo as labels corretamente
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token (e.g., [CLS], [SEP])
            elif word_idx != previous_word_idx:
                # Verificando se a label existe em label2id
                label = sentence_labels[word_idx]
                if label not in label2id:
                    print(f"Label {label} não encontrada em label2id!")  # Print de depuração
                label_ids.append(label2id.get(label, -1))  # Usa -1 se a label não for encontrada
            else:
                label_ids.append(label2id.get(sentence_labels[word_idx], -1) if label_all_tokens else -100)

            previous_word_idx = word_idx

        tokenized_inputs.append(tokenized_sentence)
        labels.append(label_ids)

    # Atribuindo as labels ao tokenized_inputs
    for i, tokenized_sentence in enumerate(tokenized_inputs):
        tokenized_sentence["labels"] = labels[i]

    # Retorna os dados tokenizados e alinhados com as labels
    return {"input_ids": [x["input_ids"] for x in tokenized_inputs],
            "attention_mask": [x["attention_mask"] for x in tokenized_inputs],
            "labels": labels}

# Aplicar a tokenização a todas as partições de dados
for i in range(10):
    print(f"Tokenizando partição {i}...")
    particoes[f"particao_{i}"] = particoes[f"particao_{i}"].map(tokenize_and_align_labels, batched=True)

print("Tokenização aplicada com sucesso!")


dict_keys([])
Verificando partição: ../models/data_division_0
Verificando partição: ../models/data_division_1
Verificando partição: ../models/data_division_2
Verificando partição: ../models/data_division_3
Verificando partição: ../models/data_division_4
Verificando partição: ../models/data_division_5
Verificando partição: ../models/data_division_6
Verificando partição: ../models/data_division_7
Verificando partição: ../models/data_division_8
Verificando partição: ../models/data_division_9
Partições carregadas: ['particao_0', 'particao_1', 'particao_2', 'particao_3', 'particao_4', 'particao_5', 'particao_6', 'particao_7', 'particao_8', 'particao_9']
Label2ID mapping: {'O': 0, 'B-LOCAL': 1, 'I-LOCAL': 2, 'B-ORGANIZACAO': 3, 'I-ORGANIZACAO': 4, 'B-PESSOA': 5, 'I-PESSOA': 6, 'B-TEMPO': 7, 'I-TEMPO': 8}
Tokenizando partição 0...
Tokenizando partição 1...
Tokenizando partição 2...
Tokenizando partição 3...
Tokenizando partição 4...
Tokenizando partição 5...
Tokenizando partição 6...
Tokeniza

## Fine-tuning the model

In [2]:
def criar_modelo(num_labels, attn_dropout, hidden_dropout):
    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        num_labels=num_labels,
        hidden_dropout_prob=hidden_dropout,
        attention_probs_dropout_prob=attn_dropout
    )
    return model

In [3]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

def objective(trial, train_set, val_set, num_labels):
    # Amostragem de hiperparâmetros
    lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    epochs = trial.suggest_categorical("epochs", [2, 3, 4, 8, 16, 32, 64, 128])
    attn_dropout = trial.suggest_uniform("attention_dropout", 0.0, 0.5)
    hidden_dropout = trial.suggest_uniform("hidden_dropout", 0.0, 0.5)
    weight_decay = trial.suggest_uniform("weight_decay", 0.01, 0.05)

    training_args = TrainingArguments(
        output_dir="./optuna_results",
        evaluation_strategy="epoch",
        save_strategy="no",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        logging_dir="./optuna_logs",
        logging_steps=10,
        report_to="none",
    )

    model = criar_modelo(num_labels, attn_dropout, hidden_dropout)

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator= data_collator,
        train_dataset=train_set,
        eval_dataset=val_set,
        compute_metrics=compute_metrics
    )

    trainer.train()
    val_metrics = trainer.evaluate()

    return val_metrics["eval_f1_macro"]  # Otimização baseada no F1 macro


In [4]:
import os
import torch
import pandas as pd
import random
import numpy as np
from torch.utils.data import Subset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification
import optuna
from optuna.samplers import TPESampler

# Garantir que os diretórios existam
checkpoints_dir = "./checkpoints"
logs_dir = "./logs_final"
final_results_dir = "./final_results"
optuna_db = "sqlite:///optuna_study.db"

os.makedirs(checkpoints_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)
os.makedirs(final_results_dir, exist_ok=True)

torch.cuda.empty_cache()

num_particoes = 10

# Função para obter uma amostra de 0.1% dos dados
def get_subset(dataset, fraction=1):
    total_size = len(dataset)
    subset_size = int(total_size * fraction)
    indices = random.sample(range(total_size), subset_size)
    return Subset(dataset, indices)

# Função para computar as métricas
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    pred_labels = []

    # Considerando apenas as classes com prefixo 'B-'
    for pred, label in zip(predictions, labels):
        true_labels.extend([l for l in label if l != -100 and l % 2 == 1])  # Apenas 'B-...'
        pred_labels.extend([p for p, l in zip(pred, label) if l != -100 and l % 2 == 1])  # Apenas 'B-...'

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average=None, labels=[1, 3, 5, 7])  # 'B-...'
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(true_labels, pred_labels, average="macro")
    accuracy = accuracy_score(true_labels, pred_labels)

    return {
        "precision_per_class": precision.tolist(),
        "recall_per_class": recall.tolist(),
        "f1_per_class": f1.tolist(),
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "accuracy": accuracy
    }

# Carregar progresso se houver checkpoints
test_results = []
completed_iterations = set()

checkpoint_file = os.path.join(checkpoints_dir, "completed_iterations.txt")
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        completed_iterations = set(map(int, f.read().splitlines()))

num_labels = len(set(label for dataset in particoes.values() for sentence in dataset["sentences"] for label in sentence["labels"]))

import torch
import optuna
import os
import numpy as np
from transformers import TrainingArguments, Trainer
from optuna.samplers import TPESampler






In [5]:


# Definição da partição de validação
validacao_set = particoes["particao_7"]

# Definir conjunto de treino excluindo a partição de validação
train_sets_validacao = [particoes[f"particao_{j}"] for j in range(num_particoes) if j != 7]
train_set_validacao = torch.utils.data.ConcatDataset(train_sets_validacao)

# Otimização de hiperparâmetros com Optuna na partição 7
study = optuna.create_study(direction="maximize", sampler=TPESampler(), storage=optuna_db, load_if_exists=True)
study.optimize(lambda trial: objective(trial, train_set_validacao, validacao_set, num_labels), n_trials=25)

best_hyperparams = study.best_params
print(f"Melhores hiperparâmetros na partição de validação: {best_hyperparams}")




[I 2025-04-15 15:47:49,949] A new study created in RDB with name: no-name-37d411c4-b779-4df6-abe7-8133fa4f96ed
/tmp/ipykernel_898209/5382319.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/tmp/ipykernel_898209/5382319.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  attn_dropout = trial.suggest_uniform("attention_dropout", 0.0, 0.5)
/tmp/ipykernel_898209/5382319.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  hidden_dropout = trial.suggest_uniform("hidden_dropout", 0.0, 0.5)


Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.078300,0.087848,"[0.9599322799097065, 0.9575971731448764, 0.980246913580247, 1.0]","[0.932054794520548, 0.8053491827637445, 0.9508982035928144, 0.8267898383371824]","[0.9457881567973311, 0.8748991121872478, 0.9653495440729484, 0.9051833122629582]",0.433086,0.390566,0.410136,0.901487
2,0.099500,0.085451,"[0.9686431014823261, 0.9631490787269682, 0.9842041312272175, 1.0]","[0.930958904109589, 0.8543833580980683, 0.9700598802395209, 0.8452655889145496]","[0.949427214305672, 0.905511811023622, 0.97708082026538, 0.9161451814768461]",0.435111,0.400074,0.416463,0.916091


/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-04-15 15:51:59,264] Trial 0 finished with value: 0.41646278078572446 and parameters: {'learning_rate': 1.7406466235278702e-05, 'batch_size': 8, 'epochs': 2, 'attention_dropout': 0.3693844027001002, 'hidden_dropout': 0.12589385497433386, 'weight_decay': 0.022685568055689512}. Best is trial 0 with value: 0.41646278078572446.
/tmp/ipykernel_898209/5382319.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/tmp/ipykernel_898209/5382319.py:10: Futu

Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.395200,0.343117,"[0.7640396413402548, 0.3333333333333333, 0.9622222222222222, 0.0]","[0.8871232876712328, 0.0014858841010401188, 0.518562874251497, 0.0]","[0.8209939148073022, 0.0029585798816568047, 0.6739299610894942, 0.0]",0.257449,0.175897,0.187235,0.545141
2,0.251100,0.237423,"[0.8765625, 0.968944099378882, 0.9647058823529412, 1.0]","[0.9221917808219178, 0.23179791976225855, 0.8838323353293414, 0.17321016166281755]","[0.8987983978638184, 0.37410071942446044, 0.9225, 0.2952755905511811]",0.423357,0.245670,0.276742,0.704195
3,0.240300,0.219257,"[0.8868517548454689, 0.9693486590038314, 0.9689119170984456, 1.0]","[0.9276712328767124, 0.37592867756315007, 0.895808383233533, 0.34872979214780603]","[0.9068023567220139, 0.5417558886509636, 0.9309271935283137, 0.5171232876712328]",0.425012,0.283126,0.321845,0.755443


...

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-04-16 13:24:45,471] Trial 23 finished with value: 0.4177770381036417 and parameters: {'learning_rate': 8.643742587669375e-06, 'batch_size': 32, 'epochs': 128, 'attention_dropout': 0.13632416963134075, 'hidden_dropout': 0.06451012072781336, 'weight_decay': 0.03991023786915887}. Best is trial 9 with value: 0.4206622550092823.
/tmp/ipykernel_898209/5382319.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/tmp/ipykernel_898209/5382319.py:10: Fut

Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.243900,0.189127,"[0.8927410617551462, 0.9666666666666667, 0.9641025641025641, 1.0]","[0.903013698630137, 0.5170876671619614, 0.9005988023952096, 0.23787528868360278]","[0.8978479978207573, 0.6737657308809293, 0.9312693498452013, 0.3843283582089552]",0.424834,0.284286,0.320801,0.757037
2,0.133800,0.119694,"[0.9519675925925926, 0.9398907103825137, 0.9777777777777777, 1.0]","[0.9013698630136986, 0.7667161961367014, 0.948502994011976, 0.6235565819861432]","[0.9259780467210807, 0.8445171849427169, 0.9629179331306991, 0.7681365576102418]",0.429960,0.360016,0.389061,0.855815
3,0.107300,0.109091,"[0.9510250569476082, 0.9501779359430605, 0.9790897908979089, 1.0]","[0.915068493150685, 0.7934621099554234, 0.9532934131736527, 0.6836027713625866]","[0.9327003630270874, 0.8647773279352227, 0.9660194174757282, 0.8120713305898491]",0.431144,0.371714,0.397285,0.875199
4,0.115600,0.101067,"[0.9576416714367487, 0.9492119089316988, 0.9791921664626683, 1.0]","[0.9167123287671233, 0.8053491827637445, 0.9580838323353293, 0.7090069284064665]","[0.9367301231802911, 0.8713826366559485, 0.9685230024213075, 0.8297297297297297]",0.431783,0.376572,0.400707,0.882103
5,0.110800,0.099812,"[0.960458452722063, 0.9475465313028765, 0.9816625916870416, 1.0]","[0.9183561643835616, 0.8320950965824666, 0.9616766467065868, 0.7251732101616628]","[0.9389355742296919, 0.8860759493670886, 0.9715668481548699, 0.8406961178045516]",0.432185,0.381922,0.404142,0.890335
6,0.109800,0.098720,"[0.9583808437856328, 0.951304347826087, 0.9827373612823674, 1.0]","[0.9210958904109589, 0.812778603268945, 0.9544910179640719, 0.745958429561201]","[0.9393685386979603, 0.8766025641025641, 0.968408262454435, 0.8544973544973545]",0.432491,0.381592,0.404320,0.889007
7,0.087400,0.097568,"[0.9622209502003435, 0.9505119453924915, 0.9816849816849816, 1.0]","[0.9210958904109589, 0.8276374442793462, 0.962874251497006, 0.7528868360277137]","[0.9412094064949608, 0.8848292295472597, 0.9721886336154776, 0.8590250329380764]",0.432713,0.384944,0.406361,0.894318
8,0.095300,0.097078,"[0.9622209502003435, 0.9490662139219015, 0.9828641370869033, 1.0]","[0.9210958904109589, 0.8306092124814265, 0.9616766467065868, 0.7551963048498845]","[0.9412094064949608, 0.8858954041204438, 0.9721549636803875, 0.8605263157894737]",0.432683,0.385398,0.406643,0.894849


/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/li

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-04-16 13:30:17,681] Trial 24 finished with value: 0.4066428988983628 and parameters: {'learning_rate': 3.7477891898859655e-06, 'batch_size': 32, 'epochs': 8, 'attention_dropout': 0.21212542320300604, 'hidden_dropout': 0.2111407979766437, 'weight_decay': 0.03632192300399994}. Best is trial 9 with value: 0.4206622550092823.


Melhores hiperparâmetros na partição de validação: {'learning_rate': 2.4685937450933634e-06, 'batch_size': 32, 'epochs': 128, 'attention_dropout': 0.1785474736783021, 'hidden_dropout': 0.21249772329155847, 'weight_decay': 0.04199083343468013}


In [5]:
test_results = []

# Hiperparâmetros otimizados na partição de validação
best_hyperparams = {
    'learning_rate': 2.4685937450933634e-06,
    'batch_size': 32,
    'epochs': 128,
    'attention_dropout': 0.1785474736783021,
    'hidden_dropout': 0.21249772329155847,
    'weight_decay': 0.04199083343468013
}

for i in range(num_particoes):
    if i == 7:
        continue  # Ignorar a partição de validação
    
    test_set = particoes[f"particao_{i}"]
    train_sets = [particoes[f"particao_{j}"] for j in range(num_particoes) if j not in {i, 7}]
    
    # Concatenando os datasets de treino
    train_set = torch.utils.data.ConcatDataset(train_sets)
    train_set = get_subset(train_set, fraction=1)
    
    # Configuração de checkpoints
    checkpoint_path = os.path.join(checkpoints_dir, f"model_checkpoint_{i}")
    os.makedirs(checkpoint_path, exist_ok=True)  # Garantir que o diretório existe
    print(f"Os checkpoints serão salvos em: {checkpoint_path}")

    # Configuração do treinamento
    final_training_args = TrainingArguments(
        output_dir=checkpoint_path,
        evaluation_strategy="epoch",
        save_strategy="epoch",  # Salvar ao final de cada época
        save_total_limit=3,
        load_best_model_at_end=True,
        learning_rate=best_hyperparams["learning_rate"],
        per_device_train_batch_size=best_hyperparams["batch_size"],
        per_device_eval_batch_size=best_hyperparams["batch_size"],
        num_train_epochs=best_hyperparams["epochs"],  # Garantindo que seja maior que 1
        weight_decay=best_hyperparams["weight_decay"],
        logging_dir=logs_dir,
        logging_steps=10,
        report_to="none",
    ) 
    
    final_model = criar_modelo(num_labels, best_hyperparams["attention_dropout"], best_hyperparams["hidden_dropout"])
    
    final_trainer = Trainer(
        model=final_model,
        args=final_training_args,
        train_dataset=train_set,
        eval_dataset=test_set,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=compute_metrics
    )
    
    final_trainer.train()

    # Forçar o salvamento do modelo ao final do treinamento
    final_trainer.save_model(checkpoint_path)

    # Verificar se os arquivos foram salvos corretamente
    print(f"Arquivos salvos em {checkpoint_path}: {os.listdir(checkpoint_path)}")

    test_metrics = final_trainer.evaluate()
    test_results.append(test_metrics)

# Cálculo da média e variância das métricas finais
avg_test_results = {
    metric: np.mean([result[metric] for result in test_results]) for metric in test_results[0]
}
var_test_results = {
    metric: np.var([result[metric] for result in test_results]) for metric in test_results[0]
}

# Criar DataFrame para a tabela desejada
df = pd.DataFrame({
    "local_precision": [p["eval_precision_per_class"][0] for p in test_results],
    "local_recall": [r["eval_recall_per_class"][0] for r in test_results],
    "local_f1": [f["eval_f1_per_class"][0] for f in test_results],

    "organizacao_precision": [p["eval_precision_per_class"][1] for p in test_results],
    "organizacao_recall": [r["eval_recall_per_class"][1] for r in test_results],
    "organizacao_f1": [f["eval_f1_per_class"][1] for f in test_results],

    "pessoa_precision": [p["eval_precision_per_class"][2] for p in test_results],
    "pessoa_recall": [r["eval_recall_per_class"][2] for r in test_results],
    "pessoa_f1": [f["eval_f1_per_class"][2] for f in test_results],

    "tempo_precision": [p["eval_precision_per_class"][3] for p in test_results],
    "tempo_recall": [r["eval_recall_per_class"][3] for r in test_results],
    "tempo_f1": [f["eval_f1_per_class"][3] for f in test_results],

    "precision_macro": [p["eval_precision_macro"] for p in test_results],
    "recall_macro": [r["eval_recall_macro"] for r in test_results],
    "f1_macro": [f["eval_f1_macro"] for f in test_results],
    "accuracy": [a["eval_accuracy"] for a in test_results],
})


df.to_csv("resultados_finais_PARAMOPAMA.csv", index=False)

print(df)
print("Média das métricas:", avg_test_results)
print("Variância das métricas:", var_test_results)



df_media_variancia = pd.DataFrame({
    "local_precision_mean": [np.mean([r["eval_precision_per_class"][0] for r in test_results])],
    "local_precision_var": [np.var([r["eval_precision_per_class"][0] for r in test_results])],
    "local_recall_mean": [np.mean([r["eval_recall_per_class"][0] for r in test_results])],
    "local_recall_var": [np.var([r["eval_recall_per_class"][0] for r in test_results])],
    "local_f1_mean": [np.mean([r["eval_f1_per_class"][0] for r in test_results])],
    "local_f1_var": [np.var([r["eval_f1_per_class"][0] for r in test_results])],

    "organizacao_precision_mean": [np.mean([r["eval_precision_per_class"][1] for r in test_results])],
    "organizacao_precision_var": [np.var([r["eval_precision_per_class"][1] for r in test_results])],
    "organizacao_recall_mean": [np.mean([r["eval_recall_per_class"][1] for r in test_results])],
    "organizacao_recall_var": [np.var([r["eval_recall_per_class"][1] for r in test_results])],
    "organizacao_f1_mean": [np.mean([r["eval_f1_per_class"][1] for r in test_results])],
    "organizacao_f1_var": [np.var([r["eval_f1_per_class"][1] for r in test_results])],

    "pessoa_precision_mean": [np.mean([r["eval_precision_per_class"][2] for r in test_results])],
    "pessoa_precision_var": [np.var([r["eval_precision_per_class"][2] for r in test_results])],
    "pessoa_recall_mean": [np.mean([r["eval_recall_per_class"][2] for r in test_results])],
    "pessoa_recall_var": [np.var([r["eval_recall_per_class"][2] for r in test_results])],
    "pessoa_f1_mean": [np.mean([r["eval_f1_per_class"][2] for r in test_results])],
    "pessoa_f1_var": [np.var([r["eval_f1_per_class"][2] for r in test_results])],

    "tempo_precision_mean": [np.mean([r["eval_precision_per_class"][3] for r in test_results])],
    "tempo_precision_var": [np.var([r["eval_precision_per_class"][3] for r in test_results])],
    "tempo_recall_mean": [np.mean([r["eval_recall_per_class"][3] for r in test_results])],
    "tempo_recall_var": [np.var([r["eval_recall_per_class"][3] for r in test_results])],
    "tempo_f1_mean": [np.mean([r["eval_f1_per_class"][3] for r in test_results])],
    "tempo_f1_var": [np.var([r["eval_f1_per_class"][3] for r in test_results])],


    # Cálculo dos valores Macro
    "precision_macro_mean": [np.mean([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(4)
    ])],  # Média de todas as precisões
    "precision_macro_var": [np.var([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(4)
    ])],

    "recall_macro_mean": [np.mean([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(4)
    ])],  # Média de todos os recalls
    "recall_macro_var": [np.var([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(4)
    ])],

    "f1_macro_mean": [np.mean([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(4)
    ])],  # Média de todos os F1
    "f1_macro_var": [np.var([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(4)
    ])],

    # Cálculo de Accuracy
    "accuracy_mean": [np.mean([result["eval_accuracy"] for result in test_results])],
    "accuracy_var": [np.var([result["eval_accuracy"] for result in test_results])]
})

# Exibindo a nova tabela com as médias e variâncias
pd.set_option("display.max_columns", None)  # Mostra todas as colunas
pd.set_option("display.width", None)        # Não quebra linha
print(df_media_variancia)


# Mapeamento de rótulos para índices (como você já tem)
label_map = {
    'O': 0,                    # Fora de entidade
    'B-LOCAL': 1,               # Início da entidade DATA
    'I-LOCAL': 2,               # Interior da entidade DATA
    'B-ORGANIZACAO': 3,             # Início da entidade EVENTO
    'I-ORGANIZACAO': 4,             # Interior da entidade EVENTO
    'B-PESSOA': 5,        # Início da entidade FUNDapelido
    'I-PESSOA': 6,        # Interior da entidade FUNDapelido
    'B-TEMPO': 7,        # Início da entidade FUNDapelido
    'I-TEMPO': 8, 
}

# Gerar a tabela de erros
predictions = final_trainer.predict(test_set)
pred_labels = predictions.predictions.argmax(axis=-1)  # As predições com maior probabilidade
true_labels = predictions.label_ids

# Mapeamento inverso de índice para rótulo
index_to_label = {v: k for k, v in label_map.items()}

# Extrair as palavras e comparar com as predições
# Extrair as palavras e comparar com as predições
error_data = []
for idx, (true, pred) in enumerate(zip(true_labels, pred_labels)):
    for word_idx, (true_label, pred_label) in enumerate(zip(true, pred)):
        # Ignorar tokens com rótulo -100 (tokens de padding ou ignore)
        if true_label == -100:
            continue
        
        # Obter as labels correspondentes aos índices
        true_class = index_to_label.get(true_label, "Desconhecido")
        pred_class = index_to_label.get(pred_label, "Desconhecido")

        # Obter a palavra real a partir do ID de token
        word_id = test_set[idx]['input_ids'][word_idx]  # ID do token
        word = tokenizer.decode([word_id], skip_special_tokens=True)  # Decodifica para a palavra

        # Se a predição for errada, armazenar a palavra e as classificações
        if true_class != pred_class:
            error_data.append({
                'word': word,  # Agora 'word' é a palavra real
                'true_class': true_class,
                'pred_class': pred_class
            })

# Criar o DataFrame de erros
error_df = pd.DataFrame(error_data)

# Exibir o DataFrame com os erros cometidos
print(error_df)

# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/Paramopama_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")




Os checkpoints serão salvos em: ./checkpoints/model_checkpoint_0


/home/juliaribeiro/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision Per Class,Recall Per Class,F1 Per Class,Precision Macro,Recall Macro,F1 Macro,Accuracy
1,0.348900,0.287080,"[0.8565636087176888, 0.9885057471264368, 0.9432515337423313, 1.0]","[0.931129476584022, 0.1382636655948553, 0.8529819694868238, 0.324582338902148]","[0.8922914466737064, 0.24259520451339917, 0.8958485069191552, 0.4900900900900901]",0.420925,0.249662,0.280092,0.706458
2,0.204400,0.169629,"[0.9396648044692737, 0.9525, 0.9663865546218487, 1.0]","[0.9267217630853994, 0.612540192926045, 0.957004160887656, 0.5584725536992841]","[0.9331484049930652, 0.7455968688845401, 0.9616724738675958, 0.7166921898928025]",0.428728,0.339415,0.373012,0.835057
3,0.150100,0.127533,"[0.9530085959885387, 0.9631236442516269, 0.969187675070028, 1.0]","[0.9162534435261708, 0.7138263665594855, 0.9597780859916782, 0.6205250596658711]","[0.9342696629213483, 0.8199445983379502, 0.9644599303135889, 0.7658321060382917]",0.431702,0.356709,0.387167,0.855186
4,0.142800,0.112353,"[0.9581151832460733, 0.9226415094339623, 0.9705882352941176, 1.0]","[0.9074380165289256, 0.7861736334405145, 0.9611650485436893, 0.6634844868735084]","[0.9320882852292021, 0.8489583333333334, 0.9658536585365853, 0.7977044476327116]",0.427927,0.368696,0.393845,0.868605
5,0.119200,0.104202,"[0.9612268518518519, 0.9261363636363636, 0.9706293706293706, 1.0]","[0.9151515151515152, 0.7861736334405145, 0.9625520110957004, 0.7303102625298329]","[0.9376234829240756, 0.8504347826086956, 0.9665738161559888, 0.8441379310344828]",0.428666,0.377132,0.399863,0.880626
6,0.079500,0.103345,"[0.9607843137254902, 0.9303201506591338, 0.9719887955182073, 1.0]","[0.9179063360881543, 0.7942122186495176, 0.9625520110957004, 0.766109785202864]","[0.9388560157790927, 0.8568950563746748, 0.9672473867595819, 0.8675675675675676]",0.429233,0.382309,0.403396,0.887615
7,0.073400,0.095492,"[0.9622641509433962, 0.929368029739777, 0.9707112970711297, 1.0]","[0.9272727272727272, 0.8038585209003215, 0.9653259361997226, 0.7398568019093079]","[0.9444444444444444, 0.8620689655172413, 0.9680111265646731, 0.850480109739369]",0.429149,0.381813,0.402778,0.891529
8,0.080300,0.096893,"[0.9637305699481865, 0.9323583180987203, 0.9720670391061452, 1.0]","[0.9223140495867769, 0.819935691318328, 0.9653259361997226, 0.7947494033412887]","[0.9425675675675675, 0.8725406330196749, 0.9686847599164927, 0.8856382978723404]",0.429795,0.389147,0.407715,0.898239
9,0.079700,0.094311,"[0.9641025641025641, 0.9358974358974359, 0.9721059972105998, 1.0]","[0.9322314049586777, 0.8215434083601286, 0.9667128987517337, 0.8066825775656324]","[0.9478991596638655, 0.875, 0.9694019471488178, 0.892998678996037]",0.430234,0.391908,0.409478,0.905228
10,0.101300,0.092062,"[0.9668192219679634, 0.9341864716636198, 0.976123595505618, 1.0]","[0.931129476584022, 0.8215434083601286, 0.9639389736477115, 0.8138424821002387]","[0.9486387875385911, 0.874251497005988, 0.9699930216329379, 0.8973684210526316]",0.430792,0.392273,0.410028,0.904948


/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/li

Arquivos salvos em ./checkpoints/model_checkpoint_0: ['model.safetensors', 'checkpoint-4880', 'checkpoint-39040', 'config.json', 'training_args.bin', 'checkpoint-38735']


...

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliar

Arquivos salvos em ./checkpoints/model_checkpoint_9: ['model.safetensors', 'checkpoint-39040', 'checkpoint-5490', 'config.json', 'training_args.bin', 'checkpoint-38735']


   local_precision  local_recall  local_f1  organizacao_precision  \
0         0.971742      0.928375  0.949563               0.936620   
1         0.980102      0.927879  0.953276               0.913876   
2         0.976786      0.962463  0.969572               0.963793   
3         0.977286      0.927263  0.951617               0.946789   
4         0.974656      0.945484  0.959848               0.953099   
5         0.971429      0.940711  0.955823               0.940559   
6         0.971067      0.925862  0.947926               0.955882   
7         0.962880      0.932970  0.947689               0.959538   
8         0.968476      0.940476  0.954271               0.961039   

   organizacao_recall  organizacao_f1  pessoa_precision  pessoa_recall  \
0            0.855305        0.894118          0.974930       0.970874   
1            0.893916        0.903785          0.967702       0.958180   
2            0.846970        0.901613          0.979064       0.960145   
3            

/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/juliaribeiro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


         word     true_class pred_class
0      Coreia  B-ORGANIZACAO    B-LOCAL
1     Polônia  B-ORGANIZACAO    B-LOCAL
2    Portugal  B-ORGANIZACAO    B-LOCAL
3      Itália  B-ORGANIZACAO    B-LOCAL
4     Espanha  B-ORGANIZACAO    B-LOCAL
..        ...            ...        ...
943       até              O    B-TEMPO
944                        O    I-TEMPO
945        de              O    I-TEMPO
946      maio              O    I-TEMPO
947       dia              O    I-TEMPO

[948 rows x 3 columns]
Arquivo salvo em: /home/juliaribeiro/Paramopama_error_data.txt


In [8]:


df_media_variancia = pd.DataFrame({
    "data_precision_mean": [np.mean([result["eval_precision_per_class"][0] for result in test_results])],
    "data_precision_var": [np.var([result["eval_precision_per_class"][0] for result in test_results])],
    "data_recall_mean": [np.mean([result["eval_recall_per_class"][0] for result in test_results])],
    "data_recall_var": [np.var([result["eval_recall_per_class"][0] for result in test_results])],
    "data_f1_mean": [np.mean([result["eval_f1_per_class"][0] for result in test_results])],
    "data_f1_var": [np.var([result["eval_f1_per_class"][0] for result in test_results])],

    "evento_precision_mean": [np.mean([result["eval_precision_per_class"][1] for result in test_results])],
    "evento_precision_var": [np.var([result["eval_precision_per_class"][1] for result in test_results])],
    "evento_recall_mean": [np.mean([result["eval_recall_per_class"][1] for result in test_results])],
    "evento_recall_var": [np.var([result["eval_recall_per_class"][1] for result in test_results])],
    "evento_f1_mean": [np.mean([result["eval_f1_per_class"][1] for result in test_results])],
    "evento_f1_var": [np.var([result["eval_f1_per_class"][1] for result in test_results])],

    "fundamento_precision_mean": [np.mean([result["eval_precision_per_class"][2] for result in test_results])],
    "fundamento_precision_var": [np.var([result["eval_precision_per_class"][2] for result in test_results])],
    "fundamento_recall_mean": [np.mean([result["eval_recall_per_class"][2] for result in test_results])],
    "fundamento_recall_var": [np.var([result["eval_recall_per_class"][2] for result in test_results])],
    "fundamento_f1_mean": [np.mean([result["eval_f1_per_class"][2] for result in test_results])],
    "fundamento_f1_var": [np.var([result["eval_f1_per_class"][2] for result in test_results])],

    "local_precision_mean": [np.mean([result["eval_precision_per_class"][3] for result in test_results])],
    "local_precision_var": [np.var([result["eval_precision_per_class"][3] for result in test_results])],
    "local_recall_mean": [np.mean([result["eval_recall_per_class"][3] for result in test_results])],
    "local_recall_var": [np.var([result["eval_recall_per_class"][3] for result in test_results])],
    "local_f1_mean": [np.mean([result["eval_f1_per_class"][3] for result in test_results])],
    "local_f1_var": [np.var([result["eval_f1_per_class"][3] for result in test_results])],

    "organizacao_precision_mean": [np.mean([result["eval_precision_per_class"][4] for result in test_results])],
    "organizacao_precision_var": [np.var([result["eval_precision_per_class"][4] for result in test_results])],
    "organizacao_recall_mean": [np.mean([result["eval_recall_per_class"][4] for result in test_results])],
    "organizacao_recall_var": [np.var([result["eval_recall_per_class"][4] for result in test_results])],
    "organizacao_f1_mean": [np.mean([result["eval_f1_per_class"][4] for result in test_results])],
    "organizacao_f1_var": [np.var([result["eval_f1_per_class"][4] for result in test_results])],

    "pessoa_precision_mean": [np.mean([result["eval_precision_per_class"][5] for result in test_results])],
    "pessoa_precision_var": [np.var([result["eval_precision_per_class"][5] for result in test_results])],
    "pessoa_recall_mean": [np.mean([result["eval_recall_per_class"][5] for result in test_results])],
    "pessoa_recall_var": [np.var([result["eval_recall_per_class"][5] for result in test_results])],
    "pessoa_f1_mean": [np.mean([result["eval_f1_per_class"][5] for result in test_results])],
    "pessoa_f1_var": [np.var([result["eval_f1_per_class"][5] for result in test_results])],

    "produtodelei_precision_mean": [np.mean([result["eval_precision_per_class"][6] for result in test_results])],
    "produtodelei_precision_var": [np.var([result["eval_precision_per_class"][6] for result in test_results])],
    "produtodelei_recall_mean": [np.mean([result["eval_recall_per_class"][6] for result in test_results])],
    "produtodelei_recall_var": [np.var([result["eval_recall_per_class"][6] for result in test_results])],
    "produtodelei_f1_mean": [np.mean([result["eval_f1_per_class"][6] for result in test_results])],
    "produtodelei_f1_var": [np.var([result["eval_f1_per_class"][6] for result in test_results])],

    # Cálculo dos valores Macro
    "precision_macro_mean": [np.mean([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(7)
    ])],  # Média de todas as precisões
    "precision_macro_var": [np.var([
        np.mean([result["eval_precision_per_class"][i] for result in test_results]) for i in range(7)
    ])],

    "recall_macro_mean": [np.mean([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(7)
    ])],  # Média de todos os recalls
    "recall_macro_var": [np.var([
        np.mean([result["eval_recall_per_class"][i] for result in test_results]) for i in range(7)
    ])],

    "f1_macro_mean": [np.mean([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(7)
    ])],  # Média de todos os F1
    "f1_macro_var": [np.var([
        np.mean([result["eval_f1_per_class"][i] for result in test_results]) for i in range(7)
    ])],

    # Cálculo de Accuracy
    "accuracy_mean": [np.mean([result["eval_accuracy"] for result in test_results])],
    "accuracy_var": [np.var([result["eval_accuracy"] for result in test_results])]
})

# Exibindo a nova tabela com as médias e variâncias
print(df_media_variancia)




NameError: name 'df' is not defined

In [1]:
# Mapeamento de rótulos para índices (como você já tem)
label_map = {
    'O': 0,                    # Fora de entidade
    'B-LOCAL': 1,               # Início da entidade DATA
    'I-LOCAL': 2,               # Interior da entidade DATA
    'B-ORGANIZACAO': 3,             # Início da entidade EVENTO
    'I-ORGANIZACAO': 4,             # Interior da entidade EVENTO
    'B-PESSOA': 5,        # Início da entidade FUNDapelido
    'I-PESSOA': 6,        # Interior da entidade FUNDapelido
    'B-TEMPO': 7,        # Início da entidade FUNDapelido
    'I-TEMPO': 8, 
}

# Gerar a tabela de erros
predictions = final_trainer.predict(test_set)
pred_labels = predictions.predictions.argmax(axis=-1)  # As predições com maior probabilidade
true_labels = predictions.label_ids

# Mapeamento inverso de índice para rótulo
index_to_label = {v: k for k, v in label_map.items()}

# Extrair as palavras e comparar com as predições
# Extrair as palavras e comparar com as predições
error_data = []
for idx, (true, pred) in enumerate(zip(true_labels, pred_labels)):
    for word_idx, (true_label, pred_label) in enumerate(zip(true, pred)):
        # Ignorar tokens com rótulo -100 (tokens de padding ou ignore)
        if true_label == -100:
            continue
        
        # Obter as labels correspondentes aos índices
        true_class = index_to_label.get(true_label, "Desconhecido")
        pred_class = index_to_label.get(pred_label, "Desconhecido")

        # Obter a palavra real a partir do ID de token
        word_id = test_set[idx]['input_ids'][word_idx]  # ID do token
        word = tokenizer.decode([word_id], skip_special_tokens=True)  # Decodifica para a palavra

        # Se a predição for errada, armazenar a palavra e as classificações
        if true_class != pred_class:
            error_data.append({
                'word': word,  # Agora 'word' é a palavra real
                'true_class': true_class,
                'pred_class': pred_class
            })

# Criar o DataFrame de erros
error_df = pd.DataFrame(error_data)

# Exibir o DataFrame com os erros cometidos
print(error_df)

# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/Ulysses_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")



NameError: name 'final_trainer' is not defined

In [8]:
# Definindo o caminho do arquivo onde o DataFrame será salvo
file_path = '/home/juliaribeiro/Ulysses_error_data.txt'

# Salvando o DataFrame em um arquivo de texto com separador de tabulação
error_df.to_csv(file_path, sep='\t', index=False)

print(f"Arquivo salvo em: {file_path}")


Arquivo salvo em: /home/juliaribeiro/Ulysses_error_data.txt
